# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-28 06:43:56] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30157, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=892839204, constrained_json_whitespace_pattern

[2025-07-28 06:43:56] Inferred chat template from model path: llama-2


[2025-07-28 06:44:08] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-28 06:44:08] Init torch distributed begin.


[2025-07-28 06:44:08] Init torch distributed ends. mem usage=0.00 GB


[2025-07-28 06:44:09] Load weight begin. avail mem=61.98 GB


...

[2025-07-28 06:44:13] Init torch distributed begin.
[2025-07-28 06:44:13] Init torch distributed ends. mem usage=0.00 GB
[2025-07-28 06:44:13] Load weight begin. avail mem=38.65 GB
[2025-07-28 06:44:14] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.21s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.21s/it]

[2025-07-28 06:44:15] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-07-28 06:44:15] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-28 06:44:15] Memory pool end. avail mem=37.40 GB


[2025-07-28 06:44:15] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB
[2025-07-28 06:44:15] INFO:     Started server process [582510]
[2025-07-28 06:44:15] INFO:     Waiting for application startup.
[2025-07-28 06:44:15] INFO:     Application startup complete.
[2025-07-28 06:44:15] INFO:     Uvicorn running on http://127.0.0.1:30157 (Press CTRL+C to quit)


[2025-07-28 06:44:16] INFO:     127.0.0.1:54108 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-28 06:44:16] INFO:     127.0.0.1:54114 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-28 06:44:16] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-28 06:45:21] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-07-28 06:45:50] INFO:     127.0.0.1:54120 - "POST /generate HTTP/1.1" 200 OK
[2025-07-28 06:45:50] The server is fired up and ready to roll!


[2025-07-28 06:45:52] INFO:     127.0.0.1:54136 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-28 06:45:52] Child process unexpectedly failed with exitcode=9. pid=583465


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-28 06:46:05] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32818, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=857585676, constrained_json_whitespace_pattern=N

[2025-07-28 06:46:05] Inferred chat template from model path: llama-2


[2025-07-28 06:46:16] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-28 06:46:16] Init torch distributed begin.


[2025-07-28 06:46:17] Init torch distributed ends. mem usage=0.00 GB


[2025-07-28 06:46:18] Load weight begin. avail mem=61.31 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]

[2025-07-28 06:46:25] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.04 GB, mem usage=0.93 GB.
[2025-07-28 06:46:25] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-28 06:46:25] Memory pool end. avail mem=36.73 GB


[2025-07-28 06:46:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=36.91 GB
[2025-07-28 06:46:25] INFO:     Started server process [590215]
[2025-07-28 06:46:25] INFO:     Waiting for application startup.
[2025-07-28 06:46:25] INFO:     Application startup complete.
[2025-07-28 06:46:25] INFO:     Uvicorn running on http://127.0.0.1:32818 (Press CTRL+C to quit)


[2025-07-28 06:46:26] INFO:     127.0.0.1:37810 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-28 06:46:26] INFO:     127.0.0.1:37818 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-28 06:46:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:46:29] INFO:     127.0.0.1:37834 - "POST /generate HTTP/1.1" 200 OK
[2025-07-28 06:46:29] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-28 06:46:31] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:46:31] INFO:     127.0.0.1:37844 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-28 06:46:31] Child process unexpectedly failed with exitcode=9. pid=590629


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-28 06:46:44] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34515, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=281561506, constrained_json_white

[2025-07-28 06:46:56] Casting torch.bfloat16 to torch.float16.


[2025-07-28 06:46:57] Casting torch.bfloat16 to torch.float16.
[2025-07-28 06:46:57] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-28 06:46:57] Init torch distributed begin.


[2025-07-28 06:46:58] Init torch distributed ends. mem usage=0.00 GB


[2025-07-28 06:46:59] Load weight begin. avail mem=44.96 GB


...

[2025-07-28 06:47:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-07-28 06:47:18] INFO:     Started server process [592154]
[2025-07-28 06:47:18] INFO:     Waiting for application startup.
[2025-07-28 06:47:18] INFO:     Application startup complete.
[2025-07-28 06:47:18] INFO:     Uvicorn running on http://127.0.0.1:34515 (Press CTRL+C to quit)


[2025-07-28 06:47:18] INFO:     127.0.0.1:52980 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-28 06:47:19] INFO:     127.0.0.1:52988 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-28 06:47:19] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:47:21] INFO:     127.0.0.1:52996 - "POST /generate HTTP/1.1" 200 OK
[2025-07-28 06:47:21] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-28 06:47:23] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:47:23] INFO:     127.0.0.1:60894 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-28 06:47:23] Child process unexpectedly failed with exitcode=9. pid=593425


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-28 06:47:34] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31339, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=466749220, constrained_json_whitespace_

[2025-07-28 06:47:44] Casting torch.bfloat16 to torch.float16.


[2025-07-28 06:47:45] Casting torch.bfloat16 to torch.float16.
[2025-07-28 06:47:45] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-28 06:47:45] Init torch distributed begin.


[2025-07-28 06:47:45] Init torch distributed ends. mem usage=0.00 GB


[2025-07-28 06:47:47] Load weight begin. avail mem=78.50 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.85it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.85it/s]

[2025-07-28 06:48:03] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=57.97 GB, mem usage=2.15 GB.
[2025-07-28 06:48:03] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-28 06:48:03] Memory pool end. avail mem=57.88 GB


[2025-07-28 06:48:04] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-28 06:48:05] INFO:     Started server process [594999]
[2025-07-28 06:48:05] INFO:     Waiting for application startup.
[2025-07-28 06:48:05] INFO:     Application startup complete.
[2025-07-28 06:48:05] INFO:     Uvicorn running on http://127.0.0.1:31339 (Press CTRL+C to quit)
[2025-07-28 06:48:05] INFO:     127.0.0.1:33166 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-28 06:48:06] INFO:     127.0.0.1:33174 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-28 06:48:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:48:08] INFO:     127.0.0.1:33176 - "POST /generate HTTP/1.1" 200 OK
[2025-07-28 06:48:08] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-28 06:48:10] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:48:10] INFO:     127.0.0.1:33182 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-28 06:48:10] Child process unexpectedly failed with exitcode=9. pid=595580


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-28 06:48:20] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=36425, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=770962237, constrained_json_whitespace_pattern=None, watchdog_timeo

[2025-07-28 06:48:32] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-28 06:48:32] Init torch distributed begin.


[2025-07-28 06:48:32] Init torch distributed ends. mem usage=0.00 GB


[2025-07-28 06:48:33] Load weight begin. avail mem=78.50 GB


[2025-07-28 06:48:33] Using model weights format ['*.safetensors']


...

[2025-07-28 06:48:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-28 06:48:39] INFO:     Started server process [596345]
[2025-07-28 06:48:39] INFO:     Waiting for application startup.
[2025-07-28 06:48:39] INFO:     Application startup complete.
[2025-07-28 06:48:39] INFO:     Uvicorn running on http://0.0.0.0:36425 (Press CTRL+C to quit)


[2025-07-28 06:48:39] INFO:     127.0.0.1:33094 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-28 06:48:40] INFO:     127.0.0.1:33108 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-28 06:48:40] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:48:41] INFO:     127.0.0.1:33114 - "POST /generate HTTP/1.1" 200 OK
[2025-07-28 06:48:41] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-28 06:48:44] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-28 06:48:45] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 11.17, #queue-req: 0, 


[2025-07-28 06:48:45] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 105.83, #queue-req: 0, 


[2025-07-28 06:48:46] Decode batch. #running-req: 1, #token: 220, token usage: 0.01, accept len: 1.48, cuda graph: False, gen throughput (token/s): 88.47, #queue-req: 0, 


[2025-07-28 06:48:47] Decode batch. #running-req: 1, #token: 285, token usage: 0.01, accept len: 1.62, cuda graph: False, gen throughput (token/s): 99.18, #queue-req: 0, 


[2025-07-28 06:48:47] Decode batch. #running-req: 1, #token: 354, token usage: 0.02, accept len: 1.73, cuda graph: False, gen throughput (token/s): 104.02, #queue-req: 0, 


[2025-07-28 06:48:48] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, accept len: 1.68, cuda graph: False, gen throughput (token/s): 99.76, #queue-req: 0, 


[2025-07-28 06:48:49] INFO:     127.0.0.1:60756 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-28 06:48:49] Child process unexpectedly failed with exitcode=9. pid=596627


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).